# 패키지 불러오기

In [1]:
import pandas as pd
import os
import re
import warnings
warnings.filterwarnings(action='ignore')
import nltk

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [2]:
os.path.realpath('.')

'C:\\Users\\user\\projects\\ml\\dacon\\영화 리뷰 감성분석'

# 데이터 로드

In [2]:
train = pd.read_csv('./data/train.csv')
train.head()

,id,document,label
0,1,영상이나 음악이 이쁘다 해도 미화시킨 불륜일뿐,0
1,2,히치콕이 이 영화를 봤다면 분명 박수를 쳤을듯...,1
2,3,괜찮은 음악영화가 또 나왔군요!!! 따뜻한 겨울이 될 것 같아요~,1
3,4,아무래도 20년도지난작품이라 지금보기는너무유치하다,0
4,5,지금까지의 영화들이 그랬듯. 이 영화역시 일본에 대한 미화는 여전하다.,0


In [8]:
train.isna().sum()

id                   0
document             0
label                0
preprocessed         0
tokenized_stem    1129
dtype: int64

In [3]:
train.shape

(5000, 3)

- 단어 임베딩을 통해 벡터화
- 모델 선정

- 어떤 임베딩 모델을 쓸 것인지
- 어떤 모델을 쓸 것인지 -> 자연어 처리에 좋은 모델 구상

# 데이터 나누기

## X, y

In [4]:
X = train['document']
y = train['label']

## train, test 데이터 나누기

In [3]:
train, val = train_test_split(train, test_size=0.33, random_state=42)

In [17]:
train.shape

(3350, 3)

In [18]:
val.shape

(1650, 3)

# 데이터 정제

In [4]:
train['preprocessed'] = train['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 한글과 공백을 제외하고 모두 제거
train['preprocessed'] = train['preprocessed'].str.replace(" +", " ") # 다중 공백 제거
train.head() # 데이터 확인

,id,document,label,preprocessed
1522,1523,극장에서 보신분들께 심심한 위로의 말을 전합니다.,0,극장에서 보신분들께 심심한 위로의 말을 전합니다
835,836,킬링타임용 그 이상 그 이하도 아님,0,킬링타임용 그 이상 그 이하도 아님
358,359,디워 만도 못한영화는 오랜 만이다,0,디워 만도 못한영화는 오랜 만이다
138,139,학생 개 답답함 스토리 어이없음마지막 학살 웃음만 나옴,0,학생 개 답답함 스토리 어이없음마지막 학살 웃음만 나옴
299,300,무료로 올레서 봣는데ㅡ화가난다....난화가나...,0,무료로 올레서 봣는데ㅡ화가난다난화가나


# 토큰화

In [5]:
from konlpy.tag import Okt 

okt = Okt() # 인스턴스 할당

In [6]:
tokenized = [] # 데이터프레임의 한 컬럼으로 추가할 리스트
for sentence in train['preprocessed']: # 전처리된 리뷰들을 하나씩 꺼내옵니다
    tokens = okt.morphs(sentence, stem = True) # 형태소 분석 (stem = True로 설정해 어간 추출을 해주었습니다)
    tokenize = " ".join(tokens) # tokens라는 리스트 안의 형태소들을 띄어쓰기로 분리된 하나의 문자열로 join시켜줍니다.
    tokenized.append(tokenize) # 형태소 단위로 띄어쓰기된 문자열을 최종 리스트에 추가해줍니다
train["tokenized_stem"] = pd.DataFrame(tokenized) # 리스트를 데이터프레임으로 변환해 tokenized_stem라는 컬럼명으로 추가해줍니다.

train.head() # 데이터 확인

,id,document,label,preprocessed,tokenized_stem
1522,1523,극장에서 보신분들께 심심한 위로의 말을 전합니다.,0,극장에서 보신분들께 심심한 위로의 말을 전합니다,최고 의 영화 울다 오다 꼭 보다 볼 까맣다 망설이다 분 들 정말 최고 이다
835,836,킬링타임용 그 이상 그 이하도 아님,0,킬링타임용 그 이상 그 이하도 아님,함 부러 악플 남다 말다 되다
358,359,디워 만도 못한영화는 오랜 만이다,0,디워 만도 못한영화는 오랜 만이다,아무 기대 없이 보다 정말 재미없다
138,139,학생 개 답답함 스토리 어이없음마지막 학살 웃음만 나옴,0,학생 개 답답함 스토리 어이없음마지막 학살 웃음만 나옴,누구 한테 하다 복수 인가 허다 접함 의 연속
299,300,무료로 올레서 봣는데ㅡ화가난다....난화가나...,0,무료로 올레서 봣는데ㅡ화가난다난화가나,박시후 한테 반한 드라마


In [7]:
train['tokenized_stem'].isna().sum()

1129

In [22]:
main_pos = [] # 데이터프레임의 새 컬럼이 될 리스트
for sentence in train['document']: # 리뷰들을 하나씩 가져옵니다
    pos = okt.pos(sentence) # 형태소 분석을 진행하고 해당 리스트를 pos라는 변수로 받습니다
    main_words = [word_pos[0] for word_pos in pos if word_pos[1] in ("Noun", "Adverb", "Adjective", "Verb")] # 가져오고자 하는 품사에 해당하면 해당 형태소를 main_words 리스트에 추가합니다.
    main_words_str = " ".join(main_words) # main_words 리스트 안의 형태소들을 띄어쓰기로 분리된 하나의 문자열로 join시켜줍니다.
    main_pos.append(main_words_str) # 선택한 형태소들로 이루어진 문자열을 최종 리스트에 추가해줍니
train["main_pos"] = pd.DataFrame(main_pos) # 리스트를 데이터프레임으로 변환해 main_pos라는 컬럼명으로 추가해줍니다.

train.head() # 데이터 확인

,id,document,label,preprocessed,main_pos
1522,1523,극장에서 보신분들께 심심한 위로의 말을 전합니다.,0,극장에서 보신분들께 심심한 위로의 말을 전합니다,최고 영화 울다 왔네요 꼭 보세요 볼 까말까 망설이시는 분 정말 최고 입니다
835,836,킬링타임용 그 이상 그 이하도 아님,0,킬링타임용 그 이상 그 이하도 아님,함 부러 악플 남기지 말아야 되겠다
358,359,디워 만도 못한영화는 오랜 만이다,0,디워 만도 못한영화는 오랜 만이다,아무 기대 없이 봤는데 정말 재미없어요
138,139,학생 개 답답함 스토리 어이없음마지막 학살 웃음만 나옴,0,학생 개 답답함 스토리 어이없음마지막 학살 웃음만 나옴,누구 하는 복수 허 접함 연속
299,300,무료로 올레서 봣는데ㅡ화가난다....난화가나...,0,무료로 올레서 봣는데ㅡ화가난다난화가나,박시후 반한 드라마


In [24]:
X_train = train.main_pos #training 데이터에서 문서 추출
y_train = train.label #training 데이터에서 라벨 추출

In [48]:
# preprocessed
val['preprocessed'] = val['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 한글과 공백을 제외하고 모두 제거
val['preprocessed'] = val['preprocessed'].str.replace(" +", " ") # 다중 공백 제거

# tokenized_stem
tokenized = []
for sentence in val['preprocessed']:
    tokens = okt.morphs(sentence, stem = True) # stem = True로 설정해 어간 추출을 해주었습니다
    tokenize = " ".join(tokens)
    tokenized.append(tokenize)
val["tokenized_stem"] = pd.DataFrame(tokenized)

# main_pos
main_pos = []
for sentence in val['document']:
    pos = okt.pos(sentence)
    main_words = [word_pos[0] for word_pos in pos if word_pos[1] in ("Noun", "Adverb", "Adjective", "Verb")]
    main_words_str = " ".join(main_words)
    main_pos.append(main_words_str)
val["main_pos"] = pd.DataFrame(main_pos)

val.head()

,id,document,label,preprocessed,tokenized_stem,main_pos
1501,1502,영화 잘봤습니다 보면서졸앗네요,0,영화 잘봤습니다 보면서졸앗네요,정말 최고다 그냥 말 을 못 하다 정도 로 최고다,정말 최고다 그냥 말 할 정도 최고다
2586,2587,대단했지 투유초콜렛 광고도 한번보삼 추억이 새록새록ㅎ,1,대단했지 투유초콜렛 광고도 한번보삼 추억이 새록새록ㅎ,NaN,NaN
2653,2654,대체 우리보고 어쩌라는거냐........좋은 점수를 줄 수가 없다,0,대체 우리보고 어쩌라는거냐좋은 점수를 줄 수가 없다,NaN,NaN
1055,1056,지금보기엔별거없음 뻔함. 극적인감동도약하고.,0,지금보기엔별거없음 뻔함 극적인감동도약하고,무언가 하다 만들다 영화 개인 적 으론 내 인생 에 터닝 포인트,무언가 하고싶게 만드는 영화 개인 내 인생 터닝 포인트
705,706,십광구네요 정말 굳입니다 추천!,1,십광구네요 정말 굳입니다 추천,평점 왜캐 낮다 공포 영화 톱 안 에 들어가다 ㄹㅇ 개 무서움,평점 왜캐 낮아 공포 영화 톱 안 들어감 개 무서움


In [26]:
X_val = val.main_pos #validation 데이터에서 전처리된 문서 추출
y_val = val.label #validation 데이터에서 라벨 추출

# 워드 임베딩

In [32]:
X_train

1522    최고 영화 울다 왔네요 꼭 보세요 볼 까말까 망설이시는 분 정말 최고 입니다
835                            함 부러 악플 남기지 말아야 되겠다
358                          아무 기대 없이 봤는데 정말 재미없어요
138                               누구 하는 복수 허 접함 연속
299                                     박시후 반한 드라마
                           ...                    
4426                                           NaN
466                어렸을 때 오빠 매일 같이 보던 아직 기억나는 추억 영화
3092                     당신 마음 정서 채워주는 영화 잭 바라지 안녕
3772                                           NaN
860                                쓰레기 원작 망치는구나 아주
Name: main_pos, Length: 3350, dtype: object

## CountVectorizer

In [34]:
vectorizer = CountVectorizer()

In [38]:
X_train.shape

(3350,)

In [ ]:
z

In [39]:
X_train = X_train[X_train.isna()].head()

4446    NaN
4854    NaN
3999    NaN
3615    NaN
3873    NaN
Name: main_pos, dtype: object

In [46]:
train.loc[4446, :]

id                              4447
document          진짜 사랑이란 건 이런 것일테지.
label                              1
preprocessed       진짜 사랑이란 건 이런 것일테지
main_pos                         NaN
tokenized_stem                   NaN
Name: 4446, dtype: object

In [35]:
# vectorizer = CountVectorizer()
vectorizer.fit(X_train)
# X_input = vectorizer.transform(X_train)

ValueError: np.nan is an invalid document, expected byte or unicode string.

In [28]:
X_input2 = vectorizer.transform(X_test)

In [18]:
X_input

<3x10850 sparse matrix of type '<class 'numpy.int64'>'
	with 15 stored elements in Compressed Sparse Row format>

# 모델 및 학습

## 랜덤포레스트

In [51]:
from sklearn.ensemble import RandomForestClassifier

In [52]:
clf = RandomForestClassifier(max_depth=6, random_state=42)
clf.fit(X_input, y_train)

RandomForestClassifier(max_depth=6, random_state=42)

In [53]:
pred = clf.predict(X_input2)

In [54]:
clf.score(X_input2, y_test)

0.6187878787878788

In [55]:
accuracy_score(y_test, pred)

0.6187878787878788

In [56]:
precision_score(y_test, pred)

0.92

In [57]:
recall_score(y_test, pred)

0.23086574654956085

In [58]:
f1_score(y_test, pred)

0.36910732196589774

## xgboost

In [37]:
import xgboost as xgb

In [40]:
xgb_model = xgb.XGBClassifier(n_estimators=500, learning_rate=0.1, max_depth=6)

In [41]:
xgb_model.fit(X_input, y_train)

c:\users\user\appdata\local\programs\python\python37\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:01:47] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.1, max_delta_step=0,
              max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=500, n_jobs=6,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [42]:
pred = xgb_model.predict(X_input2)

In [46]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

In [47]:
accuracy_score(y_test, pred)

0.7006060606060606

In [48]:
precision_score(y_test, pred)

0.7546218487394958

In [49]:
recall_score(y_test, pred)

0.5633626097867002

In [50]:
f1_score(y_test, pred)

0.6451149425287357

# 적용

In [59]:
train.shape

(5000, 3)

In [60]:
train.columns

Index(['id', 'document', 'label'], dtype='object')

In [63]:
X = train['document']
y = train['label']

In [64]:
xgb_model = xgb.XGBClassifier(n_estimators=500, learning_rate=0.1, max_depth=6)

In [65]:
vectorizer = CountVectorizer()
vectorizer.fit(X)
X_input = vectorizer.transform(X)

In [67]:
xgb_model.fit(X_input, y)

[14:17:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.1, max_delta_step=0,
              max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=500, n_jobs=6,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [68]:
test = pd.read_csv('./data/test.csv')
test.head()

,id,document
0,1,시간 때우기 좋은 영화 지루함
1,2,훈훈한 정이 느껴지는 영화! 가족끼리 드라마 보듯이 보면 딱~!
2,3,Childhood fantasy
3,4,멋있는 영화입니다. 잊을 수 없는!
4,5,너무 감동적이네요 펑펑 울었습니다


In [69]:
X_test = test['document']

In [70]:
X_test_input = vectorizer.transform(X_test)

In [71]:
pred = xgb_model.predict(X_test_input)

In [74]:
submission = pd.read_csv("./data/sample_submission.csv") #제출용 파일 불러오기
submission.head() #제출 파일이 잘 생성되었는지 확인

,id,label
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [75]:
submission["label"] = pred #예측 값 넣어주기
submission.head() # 데이터가 잘 들어갔는지 확인합니다.

,id,label
0,1,0
1,2,1
2,3,0
3,4,1
4,5,0


In [76]:
submission.to_csv("./submission.csv",index=False)

In [ ]:
test, test, test